# Linear Programming Problem(LPP) using Pulp
A typical **LPP** consists of:

- A linear **objective function** to maximize or minimize.
- A set of **constraints** (inequalities or equalities).
- **Non-negativity restrictions** on the decision variables.

### Problem
Maximize z = x + 2y  
Subject to  
2x + y <= 20   
-4x + 5y <= 10  
-x + 2y >= -2  
-x + 5y = 15  
x, y >= 0  

In [6]:
try:
    import pulp
except ImportError:
    print("PuLP not found. Installing now, please wait...\n")
    %pip install pulp

In [8]:
from pulp import LpMaximize, LpProblem, LpVariable, value, LpStatus

model = LpProblem(name="LPP Example", sense=LpMaximize)

#decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

#definne objective func
model += x+2*y

#add constraints
model += 2*x + y <= 20
model += -4*x + 5*y <= 10
model += -1*x + 2*y >= -2
model += -1*x + 5*y == 15

#solve using default solver(CBC)
status = model.solve()

# Step 6: Output the status and optimal solution
print(f"Solver Status   : {LpStatus[model.status]}")
print(f"Optimal x       : {x.value():.2f}")
print(f"Optimal y       : {y.value():.2f}")
print(f"Maximum Z = x+2y: {value(model.objective):.2f}")
print("\n--- Constraint Analysis (Shadow Prices & Slacks) ---")

# Step 7: Extract shadow prices (dual values) and slack/surplus
for name, constraint in model.constraints.items():
    print(f"{name:30s} | Shadow Price: {constraint.pi:6.2f} | Slack: {constraint.slack:6.2f}")

# Step 8: Reduced costs (how much objective would worsen per unit increase from 0)
print("\n--- Variable Analysis (Reduced Costs) ---")
print(f"x - Reduced Cost: {x.dj:.2f}")
print(f"y - Reduced Cost: {y.dj:.2f}")

C:\Users\s.sen\Documents\OR\Optimization-Python\venv\lib\site-packages\pulp\pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver Status   : Optimal
Optimal x       : 7.73
Optimal y       : 4.55
Maximum Z = x+2y: 16.82

--- Constraint Analysis (Shadow Prices & Slacks) ---
_C1                            | Shadow Price:   0.64 | Slack:  -0.00
_C2                            | Shadow Price:  -0.00 | Slack:  18.18
_C3                            | Shadow Price:  -0.00 | Slack:  -3.36
_C4                            | Shadow Price:   0.27 | Slack:  -0.00

--- Variable Analysis (Reduced Costs) ---
x - Reduced Cost: 0.00
y - Reduced Cost: 0.00


### Results
- Status : Optimal (Optimal solution found!)

| Constraint | Slack   | Shadow Price | Meaning |
| ---------- | ------- | ------------ | -------------------------------------------------------------------------------------- |
| C1         | `-0.00` | `+0.64`      | **Binding** — Used fully. Increasing RHS by 1 will increase Z by 0.64                 |
| C2         | `18.18` | `-0.00`      | **Not binding** — Not tight. Has extra room, so shadow price = 0                      |
| C3         | `-3.36` | `-0.00`      | **Violated?**  Slack is negative → possible numerical issue or sign flip           |
| C4         | `-0.00` | `+0.27`      | **Binding** — It's an equality and is tight. Increasing RHS by 1 → Z increases by 0.27|

**Reduced Cost**: If variable = 0 in the optimal solution, this tells how much the objective would worsen per unit if you forced it to be positive.
In our case, both x and y are non-zero and active, so reduced cost is 0.